In [2]:
import pandas as pd
from functions import load_data, clean_data, style_table

def main():
    # Load and clean data
    url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
    df = load_data(url)
    df = clean_data(df)

    # Ask the user if they want to display the table
    show_table = input("Show table? Y/N: ").strip().lower()

    if show_table == 'y':
        styled_df = style_table(df)
        display(styled_df)

if __name__ == "__main__":
    main()
